In [1]:
from qiskit import QuantumCircuit

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=3)

import time

In [2]:
# Create a quantum circuit with no parameters
qc = QuantumCircuit(2, 2)
qc.cx(0, 1)
qc.rz(0.1, 0)
qc.cx(0, 1)
qc.rz(0.2, 0)


print(qc)

          ┌─────────┐     ┌─────────┐
q_0: ──■──┤ Rz(0.1) ├──■──┤ Rz(0.2) ├
     ┌─┴─┐└─────────┘┌─┴─┐└─────────┘
q_1: ┤ X ├───────────┤ X ├───────────
     └───┘           └───┘           
c: 2/════════════════════════════════
                                     


In [3]:
t1 = time.time()
default_qc = pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

print(default_qc)

Time taken:  0.01859426498413086
OrderedDict({'rz': 1})
Number of 2-qubit gates:  0
Number of 1-qubit gates:  1
     ┌─────────┐
q_0: ┤ Rz(0.3) ├
     └─────────┘
q_1: ───────────
                
c: 2/═══════════
                


In [4]:
#Create a quantum circuit with parameters
from qiskit.circuit import Parameter

#Create parameter vector
p1 = Parameter("angle1")
p2 = Parameter("angle2")

pqc = QuantumCircuit(2, 2)
pqc.cx(0, 1)
pqc.rz(p1, 0)
pqc.cx(0, 1)
pqc.rz(p2, 0)

print(pqc)


          ┌────────────┐     ┌────────────┐
q_0: ──■──┤ Rz(angle1) ├──■──┤ Rz(angle2) ├
     ┌─┴─┐└────────────┘┌─┴─┐└────────────┘
q_1: ┤ X ├──────────────┤ X ├──────────────
     └───┘              └───┘              
c: 2/══════════════════════════════════════
                                           


In [5]:
t1 = time.time()
default_pqc = pm.run(pqc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_pqc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

print(default_pqc)

Time taken:  0.016770601272583008
OrderedDict({'cx': 2, 'rz': 2})
Number of 2-qubit gates:  2
Number of 1-qubit gates:  2
          ┌────────────┐     ┌────────────┐
q_0: ──■──┤ Rz(angle1) ├──■──┤ Rz(angle2) ├
     ┌─┴─┐└────────────┘┌─┴─┐└────────────┘
q_1: ┤ X ├──────────────┤ X ├──────────────
     └───┘              └───┘              
c: 2/══════════════════════════════════════
                                           


Testing UCC

In [6]:
import sys
sys.path.append('../')

from ucc.transpilers.ucc_defaults import UCCDefault1

ucc_transpiler = UCCDefault1()
transpiled_circuit = ucc_transpiler.run(pqc)

print(transpiled_circuit.count_ops())

print(transpiled_circuit)


OrderedDict({'rz': 2})
     ┌────────────┐┌────────────┐
q_0: ┤ Rz(angle1) ├┤ Rz(angle2) ├
     └────────────┘└────────────┘
q_1: ────────────────────────────
                                 
c: 2/════════════════════════════
                                 


In [7]:
from ucc import compile

ucc_qc, gate_counts = compile(pqc, mode="qiskit", get_gate_counts = True)

TypeError: UCCTranspiler.transpile() got an unexpected keyword argument 'target'

VQE ansatz

In [ ]:
from qiskit.circuit import ParameterVector

# Generalized Hadamard Test

num_qubits = 5
num_layers = 1

# Create parameterized circuit
params = ParameterVector('t', length=num_qubits * num_layers)
pqc = QuantumCircuit(num_qubits)

param_index = 0 
for layer in range(num_layers): 
    # Add RX rotation for each qubit 
    for qubit in range(num_qubits): 
        pqc.rz(params[param_index], qubit) 
        param_index += 1 
    # Add entangling gates (CX) in a linear chain 
    for qubit in range(num_qubits - 1): 
        pqc.cx(qubit, qubit + 1)

#Edge Impurity
param_final = ParameterVector('t_final', length=3)
pqc.rz(param_final[0], 0)
pqc.rx(param_final[1], 1)
pqc.rx(param_final[2], num_qubits - 1)

param_index = 0 
for layer in range(num_layers): 
    for qubit in range(num_qubits - 2, -1, -1):
        pqc.cx(qubit, qubit + 1)

print(pqc.count_ops())
print(pqc)


In [ ]:
import sys
sys.path.append('../')

from ucc.transpilers.ucc_defaults import UCCDefault1

ucc_transpiler = UCCDefault1()
transpiled_circuit = ucc_transpiler.run(pqc)

# print(transpiled_circuit)
print(transpiled_circuit.count_ops())


In [ ]:
from ucc import compile

ucc_qc, gate_counts = compile(pqc, mode="ucc", get_gate_counts = True)

print(gate_counts)

In [ ]:
#Substitute parameters with random values
import numpy as np

param_values = np.random.rand(len(params) + len(param_final))
param_dict = dict(zip(params, param_values[:len(params)]))
param_dict.update({param_final[0]: param_values[-1]})

pqc = pqc.assign_parameters(param_dict)

print(pqc)

In [ ]:
ucc_qc, gate_counts = compile(pqc, mode="ucc", get_gate_counts = True)

print(gate_counts)
print(ucc_qc)